# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)


## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [3]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Sandwick,GB,60.0000,-1.2500,49.82,66,40,19.57,scattered clouds
1,1,Dikson,RU,73.5069,80.5464,33.03,95,99,14.67,overcast clouds
2,2,Horki,BY,54.2861,30.9842,43.14,72,89,11.01,overcast clouds
3,3,Lasa,IT,46.6166,10.7002,51.37,87,56,0.76,broken clouds
4,4,Uyemskiy,RU,64.4743,40.8524,46.45,100,100,2.24,overcast clouds
5,5,Murgab,TM,37.4966,61.9714,82.78,12,0,7.87,clear sky
6,6,Kutum,SD,14.2000,24.6667,78.03,23,10,12.24,clear sky
7,7,Troitsko-Pechorsk,RU,62.7084,56.1964,41.45,80,100,6.20,overcast clouds
8,8,Mao,ES,39.8885,4.2658,68.56,94,20,1.14,fog
9,9,Nepa Nagar,IN,21.4667,76.4000,82.74,44,71,10.76,broken clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [4]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [5]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df =  city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
5,5,Murgab,TM,37.4966,61.9714,82.78,12,0,7.87,clear sky
6,6,Kutum,SD,14.2000,24.6667,78.03,23,10,12.24,clear sky
9,9,Nepa Nagar,IN,21.4667,76.4000,82.74,44,71,10.76,broken clouds
10,10,Magaria,NE,12.9983,8.9099,81.57,19,22,5.06,few clouds
13,13,Dalbandin,PK,28.8947,64.4101,84.43,5,0,5.59,clear sky
14,14,Otukpo,NG,7.1913,8.1329,72.41,95,98,3.80,light rain
16,16,Trincomalee,LK,8.5711,81.2335,88.43,63,100,0.87,overcast clouds
18,18,Buta,CD,2.7858,24.7300,73.94,88,100,1.63,overcast clouds
19,19,Pindiga,NG,9.9833,10.9333,74.61,86,100,7.49,overcast clouds
21,21,Hobyo,SO,5.3505,48.5268,81.75,69,23,13.15,few clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [6]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
5,5,Murgab,TM,37.4966,61.9714,82.78,12,0,7.87,clear sky
6,6,Kutum,SD,14.2000,24.6667,78.03,23,10,12.24,clear sky
9,9,Nepa Nagar,IN,21.4667,76.4000,82.74,44,71,10.76,broken clouds
10,10,Magaria,NE,12.9983,8.9099,81.57,19,22,5.06,few clouds
13,13,Dalbandin,PK,28.8947,64.4101,84.43,5,0,5.59,clear sky
14,14,Otukpo,NG,7.1913,8.1329,72.41,95,98,3.80,light rain
16,16,Trincomalee,LK,8.5711,81.2335,88.43,63,100,0.87,overcast clouds
18,18,Buta,CD,2.7858,24.7300,73.94,88,100,1.63,overcast clouds
19,19,Pindiga,NG,9.9833,10.9333,74.61,86,100,7.49,overcast clouds
21,21,Hobyo,SO,5.3505,48.5268,81.75,69,23,13.15,few clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [7]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Description"]].copy()

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Description
5,Murgab,TM,82.78,37.4966,61.9714,clear sky
6,Kutum,SD,78.03,14.2000,24.6667,clear sky
9,Nepa Nagar,IN,82.74,21.4667,76.4000,broken clouds
10,Magaria,NE,81.57,12.9983,8.9099,few clouds
13,Dalbandin,PK,84.43,28.8947,64.4101,clear sky
14,Otukpo,NG,72.41,7.1913,8.1329,light rain
16,Trincomalee,LK,88.43,8.5711,81.2335,overcast clouds
18,Buta,CD,73.94,2.7858,24.7300,overcast clouds
19,Pindiga,NG,74.61,9.9833,10.9333,overcast clouds
21,Hobyo,SO,81.75,5.3505,48.5268,few clouds


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [8]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
5,Murgab,TM,82.78,37.4966,61.9714,clear sky,
6,Kutum,SD,78.03,14.2000,24.6667,clear sky,
9,Nepa Nagar,IN,82.74,21.4667,76.4000,broken clouds,
10,Magaria,NE,81.57,12.9983,8.9099,few clouds,
13,Dalbandin,PK,84.43,28.8947,64.4101,clear sky,
14,Otukpo,NG,72.41,7.1913,8.1329,light rain,
16,Trincomalee,LK,88.43,8.5711,81.2335,overcast clouds,
18,Buta,CD,73.94,2.7858,24.7300,overcast clouds,
19,Pindiga,NG,74.61,9.9833,10.9333,overcast clouds,
21,Hobyo,SO,81.75,5.3505,48.5268,few clouds,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [9]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [10]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [11]:
# Display sample data
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
5,Murgab,TM,82.78,37.4966,61.9714,clear sky,
6,Kutum,SD,78.03,14.2000,24.6667,clear sky,
9,Nepa Nagar,IN,82.74,21.4667,76.4000,broken clouds,
10,Magaria,NE,81.57,12.9983,8.9099,few clouds,
13,Dalbandin,PK,84.43,28.8947,64.4101,clear sky,
14,Otukpo,NG,72.41,7.1913,8.1329,light rain,
16,Trincomalee,LK,88.43,8.5711,81.2335,overcast clouds,
18,Buta,CD,73.94,2.7858,24.7300,overcast clouds,
19,Pindiga,NG,74.61,9.9833,10.9333,overcast clouds,
21,Hobyo,SO,81.75,5.3505,48.5268,few clouds,


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [12]:
# Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.dropna()

# Display sample data

clean_hotel_df.head(20)



,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
5,Murgab,TM,82.78,37.4966,61.9714,clear sky,
6,Kutum,SD,78.03,14.2000,24.6667,clear sky,
9,Nepa Nagar,IN,82.74,21.4667,76.4000,broken clouds,
10,Magaria,NE,81.57,12.9983,8.9099,few clouds,
13,Dalbandin,PK,84.43,28.8947,64.4101,clear sky,
14,Otukpo,NG,72.41,7.1913,8.1329,light rain,
16,Trincomalee,LK,88.43,8.5711,81.2335,overcast clouds,
18,Buta,CD,73.94,2.7858,24.7300,overcast clouds,
19,Pindiga,NG,74.61,9.9833,10.9333,overcast clouds,
21,Hobyo,SO,81.75,5.3505,48.5268,few clouds,


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [13]:
# In case of an empty DataFrame, load the sample data provided
#clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
#clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [14]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index="City")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [15]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [16]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


# Display sample data
hotel_info[:10]


['\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Murgab</dd>\n<dt>Country</dt><dd>TM</dd>\n<dt>Current Weather</dt><dd>82.78 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Kutum</dd>\n<dt>Country</dt><dd>SD</dd>\n<dt>Current Weather</dt><dd>78.03 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Nepa Nagar</dd>\n<dt>Country</dt><dd>IN</dd>\n<dt>Current Weather</dt><dd>82.74 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Magaria</dd>\n<dt>Country</dt><dd>NE</dd>\n<dt>Current Weather</dt><dd>81.57 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Dalbandin</dd>\n<dt>Country</dt><dd>PK</dd>\n<dt>Current Weather</dt><dd>84.43 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Otukpo</dd>\n<dt>Country</dt><dd>NG</dd>\n<dt>Current Weather</dt><dd>72.41 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Trincomalee</dd>\n<dt>Co

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [17]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data

locations.head(10)

,Lat,Lng
5,37.4966,61.9714
6,14.2000,24.6667
9,21.4667,76.4000
10,12.9983,8.9099
13,28.8947,64.4101
14,7.1913,8.1329
16,8.5711,81.2335
18,2.7858,24.7300
19,9.9833,10.9333
21,5.3505,48.5268


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [19]:
# Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]


# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))